In [0]:
storage_account_name = "sa0staging0safebox0dg"
storage_account_key = "5OofEYMeqrhe2OWhdoqlS9AmlmxNbjOwx6bAinRmSrm+RlhrmxS6w+oRZ0aKQwfjtbiDLOfsKCd+rKbUkrzUpQ=="
container = "staging"

spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)
dbutils.fs.unmount('/mnt/staging')

dbutils.fs.ls(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/")
dbutils.fs.mount(
  source = f"wasbs://{container}@{storage_account_name}.blob.core.windows.net",
  mount_point = "/mnt/staging",
  extra_configs = {f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
)

/mnt/staging has been unmounted.
Out[128]: True

In [0]:
dbutils.fs.ls(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/")


Out[19]: [FileInfo(path='wasbs://onlyimages@sa0staging0safebox0dg.blob.core.windows.net/images/', name='images/', size=0, modificationTime=0)]

In [0]:
import cv2
import numpy as np
import pydicom
import easyocr
from matplotlib import pyplot as plt
import pandas as pd
import csv
from shutil import copyfile
from PIL import Image

final = '/dbfs/mnt/OtherImages/filename.csv'
temp_file = '/tmp/test.csv'
df = pd.read_csv('/dbfs/mnt/OtherImages/filename.csv')
go = False
dg = pd.DataFrame(columns=['dicom_id','CA','CT','BIP','LF','AF','Classe'])
reader = easyocr.Reader(['en'])
for i in dbutils.fs.ls("dbfs:/mnt/OtherImages/pngImages/"):
  if ("png" or "PNG") in i.path:
    path = "/"+i.path.replace(":","")
    imagename= path.split("/")[5].split('.')[0]
    bounds = reader.readtext(path)
    if(len(bounds)<50 ):
      try:
        if str(imagename) not in df['filename'].values:
            dict = {"filename":str(imagename)}
            df = df.append(dict, ignore_index = True)
            obj = BioInfo(bounds)
            list1 = obj.bioMetrics(bounds)
            list2 = [classdef(list1)]
            dg2 = pd.DataFrame({'dicom_id':[str(imagename)+'.png'],'CA': [str(list1[0])],
                      'CT' : [str(list1[1])],
                      'BIP' : [str(list1[2])],
                       'LF':[str(list1[3])],
                       
                        'Classe':list2})

            dg = pd.concat([dg, dg2], ignore_index = True, axis = 0)

            im = Image.open(path, 'r')
            image =  np.asarray(im)
            if(len(image.shape)<3):
              image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
            if(list1[1]!= None or (list1[3]!=None and list1[2]==None)):
              print("Femur + BIP only")
              cleanedpartiallyimage, mask, outimage = image_pixel_processingTest(image,reader)
              if(np.array(outimage).size==1):
                cv2.imwrite("/dbfs/mnt/OtherImages/preprocessing/"+str(imagename)+".PNG",cleanedpartiallyimage)
              else:
                cv2.imwrite("/dbfs/mnt/OtherImages/preprocessing/"+str(imagename)+".PNG",outimage)

            elif(list1[0]!=None):
              print('CA')
              cleanedpartiallyimage, mask, outimage = image_pixel_processingTest(image,reader)
              if(np.array(outimage).size!=1):
                cv2.imwrite("/dbfs/mnt/OtherImages/preprocessing/"+str(imagename)+".PNG",outimage)
                cv2.imwrite("/dbfs/mnt/OtherImages/mask/"+str(imagename)+".PNG",mask)
            elif(list1[2]!=None):
              print('CT')
              cleanedpartiallyimage, mask, outimage = image_pixel_processingTest(image,reader)
              if(np.array(outimage).size!=1):
                cv2.imwrite("/dbfs/mnt/OtherImages/preprocessing/"+str(imagename)+".PNG",outimage)
                cv2.imwrite("/dbfs/mnt/OtherImages/mask/"+str(imagename)+".PNG",mask)
            else :
              continue
            df.to_csv(temp_file)
            copyfile(temp_file, final)
            print("Image was saved Successfully")
      except:
        continue
            

go = True

#if(go):
#  update_postgres_table(dg)
  


CT
Image was saved Successfully
CT
Image was saved Successfully
Femur + BIP only
Image was saved Successfully
CT
Image was saved Successfully
Femur + BIP only
Image was saved Successfully
CA
Image was saved Successfully
Femur + BIP only
Image was saved Successfully
CT
Image was saved Successfully
CA
Image was saved Successfully
CT
Image was saved Successfully
CA
Image was saved Successfully
CA
Image was saved Successfully
CT
Image was saved Successfully

In [0]:


def find_biggest_contour(image):
    import easyocr
    try:
      img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    except :
      img=image
 
    ret, thresh = cv2.threshold(img,0,255,cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    areas = []
    for c in contours:
        area = cv2.contourArea(c)
        areas.append(area)
          
    biggest_Contour = []
    if len(areas) > 1:
        max_value = np.partition(areas, -2)[-1]
        max_index = areas.index(max_value)
        biggest_Contour = contours[max_index]
    
    return biggest_Contour


def crop_contour(img2, contour):
    rect = cv2.boundingRect(contour)
    x,y,w,h = rect
    if (x==0 and y ==0):
      croped = img2[y:y+w,x:x+h].copy()
    else:
      croped = img2[y:y+h, x:x+w].copy()
    contour = contour - contour.min(axis=0)
    mask = np.zeros(croped.shape[:2], np.uint8)
    cv2.drawContours(mask, [contour], -1, (255, 255, 255), -1, cv2.LINE_AA)

    dst = cv2.bitwise_and(croped, croped, mask=mask)
    return dst

def cleaningannotationsFUSDB1(image):

    hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    lower=np.array([70,0,20], dtype="uint8")
    higher=np.array([100,255,255], dtype="uint8")
    mask=cv2.inRange(hsv,lower,higher)

    lastmask=paddingsides(mask)

    image= cv2.inpaint(image,lastmask,10,cv2.INPAINT_NS)

    return image

def paddingsides(mask):
  newmask=mask.copy()
  for y,vy in enumerate(mask):
      for x,vx in enumerate(vy):
        if(mask[y,x]==255):
          newpoints=simpleget8points([y,x])
          for j in newpoints:
            try:
              newmask[j[0],j[1]]=255
            except:
              continue
  return newmask


def simpleget8points(point):
  liste=[]
  liste.append([point[0]-1,point[1]])
  liste.append([point[0]-1,point[1]-1])
  liste.append([point[0]-1,point[1]+1])
  liste.append([point[0]+1,point[1]])
  liste.append([point[0]+1,point[1]-1])
  liste.append([point[0]+1,point[1]+1])
  liste.append([point[0],point[1]-1])
  liste.append([point[0],point[1]+1])
  return liste



def getcenter(contours):
  """
  gets the center of the contour
  input : contours
  output : float
  """
  M = cv2.moments(contours)
  try:
    cX = int(M['m10'] /M['m00'])
    cY = int(M['m01'] /M['m00'])
    return [cX,cY]
  except:
    return [0,0]

def getaverage(contours):
  """
  gets the average of all the average distances of every point (to compare the outlier point to this average)
  input : contours
  output : float
  """
  avgavg=[]
  for j in contours:
    avgdistance=[]
    centerj=getcenter(j)
    for i in contours:
      centeri=getcenter(i)
      dx= centeri[0] - centerj[0]
      dy = centeri[1] - centerj[1]
      D = np.sqrt(dx*dx+dy*dy)
      avgdistance.append(D)
    avgavg.append((sum(avgdistance)/len(avgdistance)))
    if len(avgavg)!=0:
      return (sum(avgavg)/len(avgavg))
    else:
      return 1




def centerbound(bound):
  p0, p1, p2, p3 = bound[0]
  xcenter=(p0[0]+p2[0])/2
  ycenter=(p0[1]+p2[1])/2
  return [int(xcenter),int(ycenter)]

def getlistcloseboxes(point,bounds):
  centers=[]
  for i in bounds:
      centers.append(centerbound(i))
  usefullist=[]
  thelist=[]
  thelist.append(point)
  for index1,i in enumerate(thelist):
      for index2,j in enumerate(centers):
        dx= i[0] - j[0]
        dy = i[1] - j[1]
        D = np.sqrt(dx*dx+dy*dy)
        if(D!=0 and D<140 and  (j not in thelist)):
          thelist.append(j)
          usefullist.append(bounds[index2][0])
  return usefullist,thelist

def getthebigbox(usefulbounds):
  twopoint=[]
  xsmall=500000
  ysmall=500000
  xbig=0
  ybig=0
  for i in usefulbounds:
    for j in i:
      if(j[0]>xbig):
        xbig=j[0]
      if(j[0]<xsmall):
        xsmall=j[0]
      if(j[1]>ybig):
        ybig=j[1]
      if(j[1]<ysmall):
        ysmall=j[1]
  return (xsmall,ysmall),(xbig,ybig)

def draw_boxestest(image,reader):
    """
    it hides text (chunks) with a black box
    input : image and OCRreader
    output : image (cleaned from text)
    """
    bounds = reader.readtext(image)
#     image=cv2.imread(imagepath)
    currentlist=[]
    for ii in bounds:
      i=centerbound(ii)
      if(i not in currentlist):
        usefullist,thelist=getlistcloseboxes(i,bounds)
        usefullist.append(ii[0])
        if(len(usefullist)>=2):
          small,big=getthebigbox(usefullist)
          cv2.rectangle(image,(int(small[0]-6),int(small[1]-6)),(int(big[0]+5),int(big[1]+5)),(0,0,0),-1)
          # rancolor=(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
          for j in thelist:
            # cv2.circle(image, (j[0],j[1]), radius=0, color=rancolor, thickness=5)
            currentlist.append(j)
    return image


def cleaningannotationsDR2test1(imagetest, reader):
    """
    removes text boxes incase they are inside of the US image using OCRreader
    input : image and OCRreader
    output : image (without text)
    """
    imagetest = draw_boxestest(imagetest, reader)
    imagetest = cv2.rectangle(imagetest, (0, 0), (imagetest.shape[1], 8), (0, 0, 0), -1)
    return imagetest


def getactualcontours(contours,bin):
  """
  Removes outlier points that aren't part of the ellipse's points
  input: contours, and a parameter()
  output : contours
  """
  actualcontours=[]
  average=getaverage(contours)
  for j in contours:
    avgdistance=[]
    centerj=getcenter(j)
    for i in contours:
      centeri=getcenter(i)
      dx= centeri[0] - centerj[0]
      dy = centeri[1] - centerj[1]
      D = np.sqrt(dx*dx+dy*dy)
      avgdistance.append(D)
    if((sum(avgdistance)/len(avgdistance))<average+bin): actualcontours.append(j)
  return actualcontours

def contoursConvexHull(contours):
    """
    makes a polygon mask out of convex points only
    input : contours (points)
    output : mask
    """
    pts = []
    for i in range(0, len(contours)):
        for j in range(0, len(contours[i])):
            pts.append(contours[i][j])
    pts = np.array(pts)
    result = cv2.convexHull(pts)
    return result

def getannotationsELLIPTICtestTest(original):
    """
    Converts the calipers to a mask using the caliper colors

    input : original image (with calipers only(Text and sides cleaned/cropped))
    output : mask of the calipers
    """
    # Firstly, puts rectangles on the top and leftside, since there's usually a scale on the left, and to make sure there's no top(anonymization)
    cv2.rectangle(original, (0, 0), (1200, 10), (0, 0, 0), -1)
    cv2.rectangle(original, (0, 0), (47, 1200), (0, 0, 0), -1)
    # Secondly, locate the caliper color using HSV color range, and use inRange to select only that color (link to help : https://i.stack.imgur.com/gyuw4.png)
    image = cv2.cvtColor(original, cv2.COLOR_BGR2HSV)
    lower = np.array([85, 0, 20], dtype="uint8")
    upper = np.array([95, 255, 255], dtype="uint8")
    mygray = cv2.inRange(image, lower, upper)
    
    low_green = np.array([25, 52, 72])
    high_green = np.array([102, 255, 255])
    mask_green=cv2.inRange(image,low_green,high_green)
    ############
    low_white = np.array([0,0,255-15])
    high_white = np.array([255,15,255])
    mask_white=cv2.inRange(image,low_white,high_white)
    
    if(mygray.any()):
      
      # Using Gaussian and Canny filters to make it easy to get the contours
      blurred = cv2.GaussianBlur(mygray, (5, 5), 0)
      imageCanny = cv2.Canny(blurred, 0, 100, 0)
      actualcontours, hierarchy = cv2.findContours(imageCanny, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
      # Removing side contour points
      actualcontours = getactualcontours(actualcontours, 60)

      empty = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
      # Removes linear contour points
      ConvexHullPoints = contoursConvexHull(actualcontours)
      # Converts the points to Ellipse using fitEllipse
      test41 = cv2.polylines(empty, [ConvexHullPoints], True, (255, 255, 255), 3)
      imageCannyee = cv2.Canny(test41, 0, 100, 0)
      contours, hierarchy = cv2.findContours(imageCannyee, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
      for cont in contours:

        if len(cont) < 20:
          break
        elps = cv2.fitEllipse(cont)
      anotherempty = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
      # Drawing the ellipse into the empty mask
      cv2.ellipse(anotherempty, elps, (255, 255, 255), 2)
      return anotherempty   

    if(mask_white.any()):
      blurred = cv2.GaussianBlur(mask_white, (5, 5), 0)
      imageCanny = cv2.Canny(blurred, 0, 100, 0)
      actualcontours, hierarchy = cv2.findContours(imageCanny, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
      # Removing side contour points
      actualcontours = getactualcontours(actualcontours, 60)

      empty = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
      # Removes linear contour points
      ConvexHullPoints = contoursConvexHull(actualcontours)
      # Converts the points to Ellipse using fitEllipse
      test41 = cv2.polylines(empty, [ConvexHullPoints], True, (255, 255, 255), 3)
      imageCannyee = cv2.Canny(test41, 0, 100, 0)
      contours, hierarchy = cv2.findContours(imageCannyee, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
      for cont in contours:
        if len(cont) < 20:
          break
        elps = cv2.fitEllipse(cont)
      anotherempty = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
      # Drawing the ellipse into the empty mask
      cv2.ellipse(anotherempty, elps, (255, 255, 255), 2)
      return anotherempty  
    
    if(mask_green.any()):
      blurred = cv2.GaussianBlur(mask_green, (5, 5), 0)
      imageCanny = cv2.Canny(blurred, 0, 100, 0)
      actualcontours, hierarchy = cv2.findContours(imageCanny, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
      # Removing side contour points
      actualcontours = getactualcontours(actualcontours, 60)

      empty = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
      # Removes linear contour points
      ConvexHullPoints = contoursConvexHull(actualcontours)
      # Converts the points to Ellipse using fitEllipse
      test41 = cv2.polylines(empty, [ConvexHullPoints], True, (255, 255, 255), 3)
      imageCannyee = cv2.Canny(test41, 0, 100, 0)
      contours, hierarchy = cv2.findContours(imageCannyee, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
      for cont in contours:
        if len(cont) < 20:
          break
        elps = cv2.fitEllipse(cont)
      anotherempty = np.zeros((image.shape[0], image.shape[1], 3), np.uint8)
      # Drawing the ellipse into the empty mask
      cv2.ellipse(anotherempty, elps, (255, 255, 255), 2)
      return anotherempty 
    


#def cleaningannotationsDR2test2(imagetest):
#    """
#    detects where the calipers are and does inpaint process to remove it
#    input : image
#    output : image (cleaned)
#    """
#    hsv=cv2.cvtColor(imagetest,cv2.COLOR_BGR2HSV)
#    lower=np.array([70,0,20], dtype="uint8")
#    higher=np.array([100,255,255], dtype="uint8")
#    mask=cv2.inRange(hsv,lower,higher)
#
#    imagetest= cv2.inpaint(imagetest,paddingsides(mask),10,cv2.INPAINT_NS)
#    return imagetest

def bigCrop(imagetest):
    """
    finds the biggest contour (US part only) and crops the rest
    input : image
    output : image (withoutsides)
    """
    biggest_contour = find_biggest_contour(imagetest)
    imagetest = crop_contour(imagetest, biggest_contour)
    return imagetest
  
def cleaningannotations(image):
    """
    detects where the calipers are and does inpaint process to remove it
    input : image
    output : image (cleaned)
    
    """
    ########
    hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    lower_yellow=np.array([70,0,20], dtype="uint8")
    higher_yellow=np.array([100,255,255], dtype="uint8")
    mask_yellow=cv2.inRange(hsv,lower_yellow,higher_yellow)

    
    ########
    low_green = np.array([25, 52, 72])
    high_green = np.array([102, 255, 255])
    mask_green=cv2.inRange(hsv,low_green,high_green)
    ############
    low_white = np.array([0,0,255-15])
    high_white = np.array([255,15,255])
    mask_white=cv2.inRange(hsv,low_white,high_white)
    
    if (mask_yellow.any()):
      image= cv2.inpaint(image,paddingsides(mask_yellow),10,cv2.INPAINT_NS)
      return image
    
    if (mask_green.any()):
      image = cv2.inpaint(image,paddingsides(mask_green),10,cv2.INPAINT_NS)
      return image
    
    if(mask_white.any()):
      image = cv2.inpaint(image,paddingsides(mask_white),10,cv2.INPAINT_NS)
      return image

    

def image_pixel_processingTest(pixel_array_numpy,reader):
    cropimage=bigCrop(pixel_array_numpy)
    cleanedpartiallyimage=cleaningannotationsDR2test1(cropimage,reader)
    if(cropimage.shape[0]<600 and cropimage.shape[1]<800):
      cleanedpartiallyimage = cv2.resize(cleanedpartiallyimage, (800,600), interpolation = cv2.INTER_AREA)
      image=getannotationsELLIPTICtestTest(cleanedpartiallyimage)
    else:
      image=getannotationsELLIPTICtestTest(cleanedpartiallyimage)
    #cleanedimage=cleaningannotationsDR2test2(cleanedpartiallyimage)
    cleanedimage =cleaningannotations(cleanedpartiallyimage)
  
      
    return cleanedpartiallyimage,image,cleanedimage


In [0]:
!pip install --upgrade pip

Requirement already satisfied: pip in /databricks/python3/lib/python3.8/site-packages (21.0.1)
Collecting pip
 Downloading pip-22.0.4-py3-none-any.whl (2.1 MB)
▏ | 10 kB 33.1 MB/s eta 0:00:01 |▎ | 20 kB 8.4 MB/s eta 0:00:01 |▌ | 30 kB 8.1 MB/s eta 0:00:01 |▋ | 40 kB 4.3 MB/s eta 0:00:01 |▊ | 51 kB 4.4 MB/s eta 0:00:01 |█ | 61 kB 5.2 MB/s eta 0:00:01 |█ | 71 kB 5.3 MB/s eta 0:00:01 |█▎ | 81 kB 4.2 MB/s eta 0:00:01 |█▍ | 92 kB 4.7 MB/s eta 0:00:01 |█▌ | 102 kB 5.0 MB/s eta 0:00:01 |█▊ | 112 kB 5.0 MB/s eta 0:00:01 |█▉ | 122 kB 5.0 MB/s eta 0:00:01 |██ | 133 kB 5.0 MB/s eta 0:00:01 |██▏ | 143 kB 5.0 MB/s eta 0:00:01 |██▎ | 153 kB 5.0 MB/s eta 0:00:01 |██▌ | 163 kB 5.0 MB/s eta 0:00:01 |██▋ | 174 kB 5.0 MB/s eta 0:00:01 |██▊ | 184 kB 5.0 MB/s eta 0:00:01 |███ | 194 kB 5.0 MB/s eta 0:00:01 |███ | 204 kB 5.0 MB/s eta 0:00:01 |███▎ | 215 kB 5.0 MB/s eta 0:00:01 |███▍ | 225 kB 5.0 MB/s eta 0:00:01 |███▌ | 235 kB 5.0 MB/s eta 0:00:01 |███▊ | 245 kB 5.0 MB/s eta 0:00:01 |███▉ | 256 kB 5.0 MB/s eta 0:00:01 |████ | 266 kB 5.0 MB/s eta 0:00:01 |████▏ | 276 kB 5.0 MB/s eta 0:00:01 |████▎ | 286 kB 5.0 MB/s eta 0:00:01 |████▌ | 296 kB 5.0 MB/s eta 0:00:01 |████▋ | 307 kB 5.0 MB/s eta 0:00:01 |████▉ | 317 kB 5.0 MB/s eta 0:00:01 |█████ | 327 kB 5.0 MB/s eta 0:00:01 |█████ | 337 kB 5.0 MB/s eta 0:00:01 |█████▎ | 348 kB 5.0 MB/s eta 0:00:01 |█████▍ | 358 kB 5.0 MB/s eta 0:00:01 |█████▌ | 368 kB 5.0 MB/s eta 0:00:01 |█████▊ | 378 kB 5.0 MB/s eta 0:00:01 |█████▉ | 389 kB 5.0 MB/s eta 0:00:01 |██████ | 399 kB 5.0 MB/s eta 0:00:01 |██████▏ | 409 kB 5.0 MB/s eta 0:00:01 |██████▎ | 419 kB 5.0 MB/s eta 0:00:01 |██████▌ | 430 kB 5.0 MB/s eta 0:00:01 |██████▋ | 440 kB 5.0 MB/s eta 0:00:01 |██████▉ | 450 kB 5.0 MB/s eta 0:00:01 |███████ | 460 kB 5.0 MB/s eta 0:00:01 |███████ | 471 kB 5.0 MB/s eta 0:00:01 |███████▎ | 481 kB 5.0 MB/s eta 0:00:01 |███████▍ | 491 kB 5.0 MB/s eta 0:00:01 |███████▋ | 501 kB 5.0 MB/s eta 0:00:01 |███████▊ | 512 kB 5.0 MB/s eta 0:00:01 |███████▉ | 522 kB 5.0 MB/s eta 0:00:01 |████████ | 532 kB 5.0 MB/s eta 0:00:01 |████████▏ | 542 kB 5.0 MB/s eta 0:00:01 |████████▎ | 552 kB 5.0 MB/s eta 0:00:01 |████████▌ | 563 kB 5.0 MB/s eta 0:00:01 |████████▋ | 573 kB 5.0 MB/s eta 0:00:01 |████████▉ | 583 kB 5.0 MB/s eta 0:00:01 |█████████ | 593 kB 5.0 MB/s eta 0:00:01 |█████████ | 604 kB 5.0 MB/s eta 0:00:01 |█████████▎ | 614 kB 5.0 MB/s eta 0:00:01 |█████████▍ | 624 kB 5.0 MB/s eta 0:00:01 |█████████▋ | 634 kB 5.0 MB/s eta 0:00:01 |█████████▊ | 645 kB 5.0 MB/s eta 0:00:01 |█████████▉ | 655 kB 5.0 MB/s eta 0:00:01 |██████████ | 665 kB 5.0 MB/s eta 0:00:01 |██████████▏ | 675 kB 5.0 MB/s eta 0:00:01 |██████████▍ | 686 kB 5.0 MB/s eta 0:00:01 |██████████▌ | 696 kB 5.0 MB/s eta 0:00:01 |██████████▋ | 706 kB 5.0 MB/s eta 0:00:01 |██████████▉ | 716 kB 5.0 MB/s eta 0:00:01 |███████████ | 727 kB 5.0 MB/s eta 0:00:01 |███████████ | 737 kB 5.0 MB/s eta 0:00:01 |███████████▎ | 747 kB 5.0 MB/s eta 0:00:01 |███████████▍ | 757 kB 5.0 MB/s eta 0:00:01 |███████████▋ | 768 kB 5.0 MB/s eta 0:00:01 |███████████▊ | 778 kB 5.0 MB/s eta 0:00:01 |███████████▉ | 788 kB 5.0 MB/s eta 0:00:01 |████████████ | 798 kB 5.0 MB/s eta 0:00:01 |████████████▏ | 808 kB 5.0 MB/s eta 0:00:01 |████████████▍ | 819 kB 5.0 MB/s eta 0:00:01 |████████████▌ | 829 kB 5.0 MB/s eta 0:00:01 |████████████▋ | 839 kB 5.0 MB/s eta 0:00:01 |████████████▉ | 849 kB 5.0 MB/s eta 0:00:01 |█████████████ | 860 kB 5.0 MB/s eta 0:00:01 |█████████████▏ | 870 kB 5.0 MB/s eta 0:00:01 |█████████████▎ | 880 kB 5.0 MB/s eta 0:00:01 |█████████████▍ | 890 kB 5.0 MB/s eta 0:00:01 |█████████████▋ | 901 kB 5.0 MB/s eta 0:00:01 |█████████████▊ | 911 kB 5.0 MB/s eta 0:00:01 |█████████████▉ | 921 kB 5.0 MB/s eta 0:00:01 |██████████████ | 931 kB 5.0 MB/s eta 0:00:01 |██████████████▏ | 942 kB 5.0 MB/s eta 0:00:01 |██████████████▍ | 952 kB 5.0 MB/s eta 0:00:01 |██████████████▌ | 962 kB 5.0 MB/s eta 0:00:01 |██████████████▋ | 

In [0]:
!pip install opencv-python-headless==4.5.2.52
!pip install pydicom
!pip install easyocr

Collecting opencv-python-headless==4.5.2.52
 Downloading opencv_python_headless-4.5.2.52-cp38-cp38-manylinux2014_x86_64.whl (38.2 MB)
 | 10 kB 1.4 MB/s eta 0:00:28 | | 20 kB 2.3 MB/s eta 0:00:17 | | 30 kB 2.8 MB/s eta 0:00:14 | | 40 kB 3.2 MB/s eta 0:00:12 | | 51 kB 2.7 MB/s eta 0:00:15 | | 61 kB 3.2 MB/s eta 0:00:12 | | 71 kB 3.5 MB/s eta 0:00:11 | | 81 kB 3.9 MB/s eta 0:00:10 | | 92 kB 4.2 MB/s eta 0:00:10 | | 102 kB 4.6 MB/s eta 0:00:09 | | 112 kB 4.6 MB/s eta 0:00:09 | | 122 kB 4.6 MB/s eta 0:00:09 |▏ | 133 kB 4.6 MB/s eta 0:00:09 |▏ | 143 kB 4.6 MB/s eta 0:00:09 |▏ | 153 kB 4.6 MB/s eta 0:00:09 |▏ | 163 kB 4.6 MB/s eta 0:00:09 |▏ | 174 kB 4.6 MB/s eta 0:00:09 |▏ | 184 kB 4.6 MB/s eta 0:00:09 |▏ | 194 kB 4.6 MB/s eta 0:00:09 |▏ | 204 kB 4.6 MB/s eta 0:00:09 |▏ | 215 kB 4.6 MB/s eta 0:00:09 |▏ | 225 kB 4.6 MB/s eta 0:00:09 |▏ | 235 kB 4.6 MB/s eta 0:00:09 |▏ | 245 kB 4.6 MB/s eta 0:00:09 |▏ | 256 kB 4.6 MB/s eta 0:00:09 |▎ | 266 kB 4.6 MB/s eta 0:00:09 |▎ | 276 kB 4.6 MB/s eta 0:00:09 |▎ | 286 kB 4.6 MB/s eta 0:00:09 |▎ | 296 kB 4.6 MB/s eta 0:00:09 |▎ | 307 kB 4.6 MB/s eta 0:00:09 |▎ | 317 kB 4.6 MB/s eta 0:00:09 |▎ | 327 kB 4.6 MB/s eta 0:00:09 |▎ | 337 kB 4.6 MB/s eta 0:00:09 |▎ | 348 kB 4.6 MB/s eta 0:00:09 |▎ | 358 kB 4.6 MB/s eta 0:00:09 |▎ | 368 kB 4.6 MB/s eta 0:00:09 |▎ | 378 kB 4.6 MB/s eta 0:00:09 |▎ | 389 kB 4.6 MB/s eta 0:00:09 |▍ | 399 kB 4.6 MB/s eta 0:00:09 |▍ | 409 kB 4.6 MB/s eta 0:00:09 |▍ | 419 kB 4.6 MB/s eta 0:00:09 |▍ | 430 kB 4.6 MB/s eta 0:00:09 |▍ | 440 kB 4.6 MB/s eta 0:00:09 |▍ | 450 kB 4.6 MB/s eta 0:00:09 |▍ | 460 kB 4.6 MB/s eta 0:00:09 |▍ | 471 kB 4.6 MB/s eta 0:00:09 |▍ | 481 kB 4.6 MB/s eta 0:00:09 |▍ | 491 kB 4.6 MB/s eta 0:00:09 |▍ | 501 kB 4.6 MB/s eta 0:00:09 |▍ | 512 kB 4.6 MB/s eta 0:00:09 |▍ | 522 kB 4.6 MB/s eta 0:00:09 |▌ | 532 kB 4.6 MB/s eta 0:00:09 |▌ | 542 kB 4.6 MB/s eta 0:00:09 |▌ | 552 kB 4.6 MB/s eta 0:00:09 |▌ | 563 kB 4.6 MB/s eta 0:00:09 |▌ | 573 kB 4.6 MB/s eta 0:00:09 |▌ | 583 kB 4.6 MB/s eta 0:00:09 |▌ | 593 kB 4.6 MB/s eta 0:00:09 |▌ | 604 kB 4.6 MB/s eta 0:00:09 |▌ | 614 kB 4.6 MB/s eta 0:00:09 |▌ | 624 kB 4.6 MB/s eta 0:00:09 |▌ | 634 kB 4.6 MB/s eta 0:00:09 |▌ | 645 kB 4.6 MB/s eta 0:00:09 |▌ | 655 kB 4.6 MB/s eta 0:00:09 |▋ | 665 kB 4.6 MB/s eta 0:00:09 |▋ | 675 kB 4.6 MB/s eta 0:00:09 |▋ | 686 kB 4.6 MB/s eta 0:00:09 |▋ | 696 kB 4.6 MB/s eta 0:00:09 |▋ | 706 kB 4.6 MB/s eta 0:00:09 |▋ | 716 kB 4.6 MB/s eta 0:00:09 |▋ | 727 kB 4.6 MB/s eta 0:00:09 |▋ | 737 kB 4.6 MB/s eta 0:00:09 |▋ | 747 kB 4.6 MB/s eta 0:00:09 |▋ | 757 kB 4.6 MB/s eta 0:00:09 |▋ | 768 kB 4.6 MB/s eta 0:00:09 |▋ | 778 kB 4.6 MB/s eta 0:00:09 |▋ | 788 kB 4.6 MB/s eta 0:00:09 |▊ | 798 kB 4.6 MB/s eta 0:00:09 |▊ | 808 kB 4.6 MB/s eta 0:00:09 |▊ | 819 kB 4.6 MB/s eta 0:00:09 |▊ | 829 kB 4.6 MB/s eta 0:00:09 |▊ | 839 kB 4.6 MB/s eta 0:00:09 |▊ | 849 kB 4.6 MB/s eta 0:00:09 |▊ | 860 kB 4.6 MB/s eta 0:00:09 |▊ | 870 kB 4.6 MB/s eta 0:00:09 |▊ | 880 kB 4.6 MB/s eta 0:00:09 |▊ | 890 kB 4.6 MB/s eta 0:00:09 |▊ | 901 kB 4.6 MB/s eta 0:00:09 |▊ | 911 kB 4.6 MB/s eta 0:00:09 |▊ | 921 kB 4.6 MB/s eta 0:00:09 |▉ | 931 kB 4.6 MB/s eta 0:00:09 |▉ | 942 kB 4.6 MB/s eta 0:00:09 |▉ | 952 kB 4.6 MB/s eta 0:00:09 |▉ | 962 kB 4.6 MB/s eta 0:00:09 |▉ | 972 kB 4.6 MB/s eta 0:00:09 |▉ | 983 kB 4.6 MB/s eta 0:00:09 |▉ | 993 kB 4.6 MB/s eta 0:00:09 |▉ | 1.0 MB 4.6 MB/s eta 0:00:09 |▉ | 1.0 MB 4.6 MB/s eta 0:00:09 |▉ | 1.0 MB 4.6 MB/s eta 0:00:09 |▉ | 1.0 MB 4.6 MB/s eta 0:00:09 |▉ | 1.0 MB 4.6 MB/s eta 0:00:09 |▉ | 1.1 MB 4.6 MB/s eta 0:00:09 |█ | 1.1 MB 4.6 MB/s eta 0:00:09 |█ | 1.1 MB 4.6 MB/s eta 0:00:09 |█ | 1.1 MB 4.6 MB/s eta 0:00:09 |█ | 1.1 MB 4.6 MB/s eta 0:00:09 |█ | 1.1 MB 4.6 MB/s eta 0:00:09 |█ | 1.1 MB 4.6 MB/s eta 0:00:09 |█ | 1.1 MB 4.6 MB/s eta 0:00:09 |█ | 1.1 MB 4.6 MB/s eta 0:00:09 |█ | 1.1 MB 4.6 MB/s eta 0:00:09 |█ | 1.2 MB 4.6 MB/s eta 0:00:09 |█ | 1.2 MB 4.6 MB/s eta 0:00:09 |█ | 1.2 M

In [0]:

            
def convertMMtoCM(value,unit):
  if(unit=="mm"):
    return str(round(value/10,2))+"cm"
  else:
    return str(round(value,2))+str(unit)

def processBioInfo(text):
  """
  corrects characters that has been mislabeled by the OCR
  """
  text=text.replace("A","4")
  text=text.replace("o","0")
  text=text.replace("O","0")
  text=text.replace("i","1")
  text=text.replace("b","6")
  text=text.replace("B","8")
  text=text.replace(" ",".") #to replace spaces with dots in cases like these : 7 74cm --> 7.74cm
  text=text.replace("#","0")
  return text


class findTheLength :
    def __init__(self,bounds , n):
        self.bounds = bounds
        self.n = n

    def findTheLengthValue(self):
        if(self.n==0):
            if(any(metric in self.bounds[self.n][1] for metric in ["cm","mm","cn","Cm"])):
              return str(self.bounds[self.n][1])
            elif(any(metric in self.bounds[self.n+1][1] for metric in ["cm","mm","cn","Cm"])):
              return str(self.bounds[self.n+1][1])
            elif(any(metric in self.bounds[self.n+2][1] for metric in ["cm","mm","cn","Cm"])):
              return str(self.bounds[self.n+2][1])
            return 1
        elif(self.n==len(self.bounds)-1):
          if(any(metric in self.bounds[self.n][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n][1])
          elif(any(metric in self.bounds[self.n-2][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n-2][1])
          elif(any(metric in self.bounds[self.n-1][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n-1][1])
          return 1
        elif(self.n==len(self.bounds)-2):
          if(any(metric in self.bounds[self.n+1][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n+1][1])
          elif(any(metric in self.bounds[self.n][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n][1])
          elif(any(metric in self.bounds[self.n-1][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n-1][1])
          elif(any(metric in self.bounds[self.n-2][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n-2][1])
          return 1
        else:
          if(any(metric in self.bounds[self.n][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n][1])
          elif(any(metric in self.bounds[self.n+1][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n+1][1])
          elif(any(metric in self.bounds[self.n+2][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n+2][1])
          elif(any(metric in self.bounds[self.n-1][1] for metric in ["cm","mm","cn","Cm"])):
            return (self.bounds[self.n-1][1])
          elif(any(metric in self.bounds[self.n-2][1] for metric in ["cm","mm","cn","Cm"])):
            return str(self.bounds[self.n-2][1])
          return 1
        

        

        
def convertTextToFloat(word):
  for i,CARACTER in enumerate(word):
      space=False
      if(CARACTER=='c' or CARACTER=='m'):
        try:
          if(word[i+1]=="m"):
            UNIT=CARACTER+word[i+1]
            if(word[i-1]==" "):
              space=True
              newWORD=word[:i-1]+word[i:]
            else:
              newWORD=word
            for j in range(-7,-3):
              try:
                if(space):
                  if(i+j<0):
                    continue
                  return float(processBioInfo(newWORD[i+j:i-1])),UNIT
                else:
                  if(i+j<0):
                    continue
                  return float(processBioInfo(newWORD[i+j:i])),UNIT
              except:
                continue
        except:
          continue

        
class BioInfo :
    def __init__(self , bounds):
        self.bounds = bounds

    def bioMetrics(self ,bounds) :
      BIO=[None,None,None,None]
      for n,i in enumerate(bounds):

                  
          if (any(metric in i[1] for metric in ["BIP","A BIP","ABIP","BIPI","BlP","A BlP","ABlP","8IP","8lP","A8IP","A8lP","A 8IP"])):
            if(i[1] in ["BIP","A BIP","ABIP","BIPI","BlP","A BlP","ABlP","8IP","8lP","A8IP","A8lP","A 8IP"]):
              FTL = findTheLength(self.bounds, n)
              bio = FTL.findTheLengthValue()
              if (bio != 1):
                  try:

                      VALUE, UNIT = convertTextToFloat(bio)
                  except:
                      continue
                  BIO[3] = convertMMtoCM(VALUE, UNIT)

          if (any(metric in i[1] for metric in ["CT","A CT","ACT",'ct',"Ct","cT"])):
              FTL = findTheLength(self.bounds, n)
              bio = FTL.findTheLengthValue()
              if (bio != 1):
                  try:
                      bio = bio.replace("n","m")
                      VALUE, UNIT = convertTextToFloat(bio)
                  except:
                      continue
                  BIO[2] = convertMMtoCM(VALUE, UNIT)

          if (any(metric in i[1] for metric in ["LF",'lf','Lf','lF',"A LF","ALF","alf","Alf","aLf","alF",'LLF','llf','lff'])):

            if(i[1] in ["LF",'lf','Lf','lF',"A LF","ALF","alf","Alf","aLf","alF",'LLF','llf','lff'] ):
              
              FTL = findTheLength(self.bounds, n)
              bio = FTL.findTheLengthValue()
              if (bio != 1):
                  try:
                      bio = bio.replace("n", "m")
                      VALUE, UNIT = convertTextToFloat(bio)
                  except:
                      continue
                  BIO[1] = convertMMtoCM(VALUE, UNIT)



          if(any(metric in i[1] for metric in ["CA","A CA","ACA","ca","Ca","cA"])):
            if(i[1] in ["CA","A CA","ACA","ca","Ca","cA"]):
              FTL = findTheLength(self.bounds,n)
              bio=FTL.findTheLengthValue()

              if(bio!=1):
                try:
                    bio = bio.replace("n", "m")
                    VALUE,UNIT=convertTextToFloat(bio)
                except:
                    continue
                BIO[0]=convertMMtoCM(VALUE,UNIT)




      return BIO



In [0]:
def classdef(measures):
  if(not measures[0]==None):
    return "Abdominal"
  if(not measures[1]==None):
    return "Femur"
  if(measures[2]!=None or measures[3]!=None):
    return "Transthalamique"
  else:
    return "Other"   

In [0]:

!sudo apt-get install sshpass
!sshpass -p "zaq1xsw2=" ssh -o "StrictHostKeyChecking no" -fNL 5555:localhost:5432 root@105.73.89.93;

# write results to postgres table
import psycopg2
DB_HOST = "localhost"
DB_NAME = "Anonymization"
DB_USER = "postgres"
DB_PASS = "c_X7F'R-6y,7LGdp"
def update_postgres_table(dataframe):
  try:
    connection = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST,port=5555)
    cursor = connection.cursor()
    #insert results of preprocessing in postgresql 
    for ind, dicom in dataframe.iterrows():
      try:
        postgreSQL_insert_Query = 'INSERT INTO "Frame" AS F ("dicom_id","extracted_AC", "extracted_HC", "extracted_BIP", "extracted_LF", "extracted_class") VALUES(%s,%s,%s,%s,%s,%s) ON CONFLICT (dicom_id) DO UPDATE SET ("extracted_AC", "extracted_HC", "extracted_BIP", "extracted_LF", "extracted_class") = (%s,%s,%s,%s,%s) WHERE F."dicom_id" = %s RETURNING *'
        cursor.execute(postgreSQL_insert_Query, (dicom['dicom_id'].split('.')[0], str(dicom['CA']), str(dicom['CT']), str(dicom['BIP']), str(dicom['LF']), str(dicom['Classe']),
                                                 dicom['CA'], str(dicom['CT']), str(dicom['BIP']), str(dicom['LF']), str(dicom['Classe']),
                                                 dicom['dicom_id'].split('.')[0]))
       # A="INSERT INTO \"Frame\" VALUES (%s,%s,%s,%s,%s,%s)",(dicom['dicom_id'].split('.')[0],dicom['CA'],dicom['CT'],dicom['BIP'],dicom['LF'],dicom['Classe'])
       # cursor.execute("INSERT INTO \"Frame\" VALUES (%s,%s,%s,%s,%s,%s)",(dicom['dicom_id'].split('.')[0],dicom['CA'],dicom['CT'],dicom['BIP'],dicom['LF'],dicom['Classe']))
        connection.commit()
        
        
      except (Exception, psycopg2.Error) as error:
        print("Error while insert to PostgreSQL", error)
  except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)
  finally:
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")    


Reading package lists... 0%Reading package lists... 0%Reading package lists... 0%Reading package lists... 6%Reading package lists... 6%Reading package lists... 6%Reading package lists... 6%Reading package lists... 60%Reading package lists... 60%Reading package lists... 60%Reading package lists... 60%Reading package lists... 69%Reading package lists... 69%Reading package lists... 75%Reading package lists... 75%Reading package lists... 80%Reading package lists... 80%Reading package lists... 80%Reading package lists... 80%Reading package lists... 81%Reading package lists... 81%Reading package lists... 81%Reading package lists... 81%Reading package lists... 88%Reading package lists... 88%Reading package lists... 93%Reading package lists... 93%Reading package lists... 95%Reading package lists... 97%Reading package lists... 97%Reading package lists... 97%Reading package lists... 97%Reading package lists... 99%Reading package lists... 99%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree 
Reading state information... 0%Reading state information... 0%Reading state information... Done
The following NEW packages will be installed:
 sshpass
0 upgraded, 1 newly installed, 0 to remove and 28 not upgraded.
Need to get 10.5 kB of archives.
After this operation, 30.7 kB of additional disk space will be used.
0% [Working] Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 sshpass amd64 1.06-1 [10.5 kB]
80% [1 sshpass 10.5 kB/10.5 kB 100%] 100% [Working] Fetched 10.5 kB in 0s (244 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package sshpass.
(Reading database ... (Reading database ... 5%(Reading database ... 10%(Reading database ... 15%(Reading database ... 20%(Reading database ... 25%(Reading database ... 30%(Reading database ... 35%(Reading database ... 40%(Reading database ... 45%(Reading database ... 50%(Reading database ... 55%(Reading database ... 60%(Reading database ... 65%(Reading database ... 70%(Reading database ... 75%(Reading database ... 80%(Reading database ... 85%(Reading database ... 90%(Reading database ... 95%(Reading database ... 100%(Reading database ... 97954 files and directories currently installed.)
Preparing to unpack .../sshpass_1.06-1_amd64.deb ...
Unpacking sshpass (1.06-1) ...
Setting up sshpass (1.06-1) ...
Processing triggers for man-db (2.9.1-1) ...
Warning: Permanently added '105.73.89.93' (ECDSA) to the list of known hosts.

In [0]:
import pandas as pd
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST,port=5555)
cr = conn.cursor()
cr.execute('SELECT * FROM "DICOM-Images";')
tmp = cr.fetchall()
col_names = []
for elt in cr.description:
    col_names.append(elt[0])
dh = pd.DataFrame(tmp, columns=col_names)
dp = dh[['dicom_id','is_video']]
dp = dp[dp["is_video"]=='true']



In [0]:
#!pip uninstall opencv-python-headless -y 
#!pip install opencv-python --upgrade


WARNING: Skipping opencv-python-headless as it is not installed.
Collecting opencv-python
 Downloading opencv_python-4.5.5.62-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.4 MB)
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 24.1 MB/s eta 0:00:00

Requirement already satisfied: numpy>=1.14.5 in /databricks/python3/lib/python3.8/site-packages (from opencv-python) (1.20.1)
Installing collected packages: opencv-python
Successfully installed opencv-python-4.5.5.62